# Usando `sqlite3` para armar base de datos

Este notebook utiliza archivos alojados en disco local, ya que por su peso era imposible subirlo al Drive de una manera más o menos "amigable". 

In [1]:
import os
import pandas as pd
import numpy as np
import csv
import sqlite3
import time


# Seteamos working directory
wd_path = "C:/Users/Asus/Desktop/CEP/Limpieza nombres de Empresas/base_aduana"
os.chdir(wd_path)

In [2]:
#Obtener headers de un csv
def header(filepath):
  with open(filepath, newline=None) as f:
    csv_reader = csv.reader(f)
    csv_headings = next(csv_reader)
  return csv_headings

In [3]:
#Lee records desde csv
def reader(abs_path): 
  with open(abs_path, newline='\n') as csvfile:
      reader = csv.reader(csvfile, quotechar='"', delimiter=',')
      lines = []
      for line in reader: 
        lines.append(tuple(line))
  return lines[1:]  

In [4]:
#Transforma python dtypes a sqlite dtypes
def py_to_sqlite_type(dtype): 
    if 'int' in dtype:
        return 'INTEGER'
    if 'float' in dtype: 
        return 'REAL'
    if 'O' in dtype or 'object' in dtype: 
        return 'TEXT'
    if 'time' in dtype or 'date' in dtype:
        return 'TEXT'

In [5]:
#Escribe query de cracion de tabla si le provees una lista de tuplas con el schema.
def table_creation_query(nombre_tabla, schema_tuples, primary_key): 
    
    col_names, pydtypes = [k for k,v in schema_tuples],[v for k,v in schema_tuples]
    
    sqlite_types = [py_to_sqlite_type(x) for x in pydtypes]
    
    if primary_key == None: 

      sqlite_create_table_query = 'CREATE TABLE '+nombre_tabla+'('
      for k,v in zip(col_names,sqlite_types):
        sqlite_create_table_query += k+(' ')+v+','

      sqlite_create_table_query = sqlite_create_table_query[:-1]+')'

    elif len(primary_key) == 1: 
                     
      sqlite_create_table_query = 'CREATE TABLE '+nombre_tabla+'('
      for k,v in zip(col_names,sqlite_types):
        if k==str(primary_key[0]):
          sqlite_create_table_query += k+(' ')+v+' PRIMARY KEY,'
        else: 
          sqlite_create_table_query += k+(' ')+v+','

      sqlite_create_table_query = sqlite_create_table_query[:-1]+')'
    
    else:
      primary_key = tuple(primary_key)
        
      sqlite_create_table_query = 'CREATE TABLE '+nombre_tabla+'('
      for k,v in zip(col_names,sqlite_types):
        sqlite_create_table_query += k+(' ')+v+','
        
      sqlite_create_table_query = sqlite_create_table_query+'PRIMARY KEY '+str(primary_key).replace("'","")+')'
    
    return sqlite_create_table_query

In [6]:
def insert_records_from_file(filepath,nombre_base,nombre_tabla, nombres_columnas):
    
    sqlite_insert_query = 'INSERT INTO '+nombre_tabla+(' ')+'\n'+str(tuple([x.lower() for x in nombres_columnas])).replace("'","")+'\n VALUES '+str(tuple(len(nombres_columnas)*['?']))
    sqlite_insert_query = sqlite_insert_query.replace("'","")
    
    t = time.time()
    try:
        sqliteConnection = sqlite3.connect(nombre_base)
        sqliteConnection.text_factory = str 
        cursor = sqliteConnection.cursor()
        print("Connected to SQLite")

        recordlist = reader(filepath)

        cursor.executemany(sqlite_insert_query, recordlist)
        sqliteConnection.commit()
        
        print("Total", cursor.rowcount, "Records inserted successfully into {} table".format(nombre_tabla))
        sqliteConnection.commit()
        cursor.close()

        print("\n Time Taken: {} sec".format(time.time()-t)) 

    except sqlite3.Error as error:
        print("Failed to insert multiple records into sqlite table", error)
    finally:
        if (sqliteConnection):
            sqliteConnection.close()
            print("The SQLite connection is closed")

In [7]:
#Crea una tabla si le provees un schema. 
def create_table(nombre_base, nombre_tabla, schema_tuples, primary_key): 
    
    create_table_query = table_creation_query(nombre_tabla, schema_tuples, primary_key)

    try:
        sqliteConnection = sqlite3.connect(nombre_base)
        
        cursor = sqliteConnection.cursor()
        print("Successfully Connected to SQLite")
        
        cursor.execute(create_table_query)
        sqliteConnection.commit()
        print("SQLite {} table created".format(nombre_tabla))

        cursor.close()
    except sqlite3.Error as error:
        print("Error while creating a sqlite table", error)
    finally:
        if (sqliteConnection):
            sqliteConnection.close()
            print("sqlite connection is closed")

In [8]:
#Función para definir query de creacion dandole un dataframe de pandas. 
def define_query(nombre_base, nombre_tabla, pandas_df, primary_key): 
    
    col_names = [x.lower() for x in pandas_df.dtypes.index]
    
    
    sqlite_types = [py_to_sqlite_type(x) for x in [str(x) for x in pandas_df.infer_objects().dtypes.values]]
    
    if primary_key == None: 

      sqlite_create_table_query = 'CREATE TABLE '+nombre_tabla+'('
      for k,v in zip(col_names,sqlite_types):
        sqlite_create_table_query += k+(' ')+v+','

      sqlite_create_table_query = sqlite_create_table_query[:-1]+')'

    elif len(primary_key) == 1: 
                     
      sqlite_create_table_query = 'CREATE TABLE '+nombre_tabla+'('
      for k,v in zip(col_names,sqlite_types):
        if k==str(primary_key[0]):
          sqlite_create_table_query += k+(' ')+v+' PRIMARY KEY,'
        else: 
          sqlite_create_table_query += k+(' ')+v+','

      sqlite_create_table_query = sqlite_create_table_query[:-1]+')'
    
    else:
      primary_key = tuple(primary_key)
        
      sqlite_create_table_query = 'CREATE TABLE '+nombre_tabla+'('
      for k,v in zip(col_names,sqlite_types):
        sqlite_create_table_query += k+(' ')+v+','
        
      sqlite_create_table_query = sqlite_create_table_query+'PRIMARY KEY '+str(primary_key).replace("'","")+')'
    
    return sqlite_create_table_query

In [9]:
#Crea tabla si le provees un dataframe de pandas
def create_table_from_pandas_df(nombre_base, nombre_tabla, pandas_df, primary_key):
    
    create_table_query = define_query(nombre_base, nombre_tabla, pandas_df, primary_key)

    try:
        sqliteConnection = sqlite3.connect(nombre_base)
        
        cursor = sqliteConnection.cursor()
        print("Successfully Connected to SQLite")
        
        cursor.execute(create_table_query)
        sqliteConnection.commit()
        print("SQLite {} table created".format(nombre_tabla))

        cursor.close()

    except sqlite3.Error as error:
        print("Error while creating a sqlite table", error)
    finally:
        if (sqliteConnection):
            sqliteConnection.close()
            print("sqlite connection is closed")

In [10]:
#Función para insertar mcuhas filas.
def insert_pandas_df_into_sqlite_table(nombre_base, nombre_tabla, pandas_df):
    
    sqlite_insert_query = 'INSERT INTO '+nombre_tabla+(' ')+'\n'+str(tuple([x.lower() for x in pandas_df.columns])).replace("'","")+'\n VALUES '+str(tuple(len(pandas_df.columns)*['?']))
    sqlite_insert_query = sqlite_insert_query.replace("'","")
    recordlist = [tuple(x) for x in pandas_df.values]
    
    try:
        sqliteConnection = sqlite3.connect(nombre_base)
        cursor = sqliteConnection.cursor()
        print("Connected to SQLite")
                
        sqliteConnection.commit()
        
             
        cursor.executemany(sqlite_insert_query, recordlist)
        sqliteConnection.commit()
        
        print("Total", cursor.rowcount, "Records inserted successfully into {} table".format(nombre_tabla))
        sqliteConnection.commit()
        cursor.close()

    except sqlite3.Error as error:
        print("Failed to insert multiple records into sqlite table", error)
    finally:
        if (sqliteConnection):
            sqliteConnection.close()
            print("The SQLite connection is closed")

#Con este codigo corremos un archivo sql que puede tener muchos scripts distintos

def run_sql_file(nombre_base, path):

    try:
        sqliteConnection = sqlite3.connect(nombre_base)
        cursor = sqliteConnection.cursor()
        print("Successfully Connected to SQLite")

        with open(path, 'r') as sqlite_file:
            sql_script = sqlite_file.read()

        cursor.executescript(sql_script)
        print("SQLite script executed successfully")
        cursor.close()

    except sqlite3.Error as error:
        print("Error while executing sqlite script", error)
    finally:
        if (sqliteConnection):
            sqliteConnection.close()
            print("sqlite connection is closed")

def retrieve_query_data(nombre_base,query_path): 
    
    try:
        sqliteConnection = sqlite3.connect(nombre_base)
        cursor = sqliteConnection.cursor()
        print("Successfully Connected to SQLite")

        with open(query_path, 'r') as sqlite_file:
          sql_script = sqlite_file.read()
        print('***'*10)
        print("Your query is:\n{}".format(sql_script))
        print('***'*10)

        cursor.execute(sql_script)
        r = cursor.fetchall()
        print("SQLite script executed successfully")
        
        rows = []
        for row in r:
            rows.append(row)
            
    except sqlite3.Error as error:
        print("Error while executing sqlite script", error)
    finally:
        if (sqliteConnection):
            sqliteConnection.close()
            print("sqlite connection is closed")
    
    return rows

def retrieve_query_keys(nombre_base,query_path):
    
    try:
        sqliteConnection = sqlite3.connect(nombre_base)
        cursor = sqliteConnection.cursor()
        print("Successfully Connected to SQLite")

        with open(query_path, 'r') as sqlite_file:
          sql_script = sqlite_file.read()
        print('***'*10)
        print("Your query is:\n{}".format(sql_script))
        print('***'*10)

        cursor.execute(sql_script)
        cols = [description[0] for description in cursor.description]
        print("SQLite script executed successfully")
        
        cursor.close()

    except sqlite3.Error as error:
        print("Error while executing sqlite script", error)
    finally:
        if (sqliteConnection):
            sqliteConnection.close()
            print("sqlite connection is closed")
    
    return cols

def retrieve_query_pandas(nombre_base, query_path, limit): 
    df = pd.DataFrame(retrieve_query_data(nombre_base,query_path), columns = retrieve_query_keys(nombre_base,query_path))
    print("The df shape is: ", df.shape)
    if limit == 'all':
      return df
    else:
      return df[:limit]

In [11]:
#Defino nombre Base
nombre_base = 'base_importaciones19062021.db'

In [15]:
#Path, nombre, tabla, dtypes de BASE SUCIA
filename_base_sucia = "base_aduana.csv"
base_sucia = 'base_sucia'
cols = header(filename_base_sucia)
print(cols)
cols[0] = "id"
print(cols)

['', 'anio', 'cuit_impor', 'rsocial', 'prove_dest', 'sim_sptos', 'porg', 'origen', 'u_estad', 'uni_estad', 'cant_est', 'kg', 'fob', 'cif']
['id', 'anio', 'cuit_impor', 'rsocial', 'prove_dest', 'sim_sptos', 'porg', 'origen', 'u_estad', 'uni_estad', 'cant_est', 'kg', 'fob', 'cif']


In [16]:
#dtypes de BASE SUCIA
dtypes = ['int','int','int','O','O','O','int','O','int','O','int','int','float','float']
schema = [(c,d) for c,d in zip(cols,dtypes)]
print(schema)

#Creo Base si no existe y tabla BASE SUCIA si no existe. 
create_table(nombre_base,base_sucia,schema,None)

#Inserto Records BASE SUCIA desde archivo. 
absolute_path = wd_path+'/'+filename_base_sucia
insert_records_from_file(absolute_path,nombre_base,base_sucia,cols)

[('id', 'int'), ('anio', 'int'), ('cuit_impor', 'int'), ('rsocial', 'O'), ('prove_dest', 'O'), ('sim_sptos', 'O'), ('porg', 'int'), ('origen', 'O'), ('u_estad', 'int'), ('uni_estad', 'O'), ('cant_est', 'int'), ('kg', 'int'), ('fob', 'float'), ('cif', 'float')]
Successfully Connected to SQLite
SQLite base_sucia table created
sqlite connection is closed
Connected to SQLite
Total 11474579 Records inserted successfully into base_sucia table

 Time Taken: 1027.365369796753 sec
The SQLite connection is closed


In [18]:
#Genero la tabla de pandas merge e insterto records

filename_pandas_merge = "pandas_merge18_06_2021.csv"
pandas_merge_df = pd.read_csv(filename_pandas_merge)
pandas_merge = "pandas_merge"
create_table_from_pandas_df(nombre_base,pandas_merge,pandas_merge_df,None)
insert_pandas_df_into_sqlite_table(nombre_base,pandas_merge, pandas_merge_df)

Successfully Connected to SQLite
SQLite pandas_merge table created
sqlite connection is closed
Connected to SQLite
Total 2303938 Records inserted successfully into pandas_merge table
The SQLite connection is closed


In [20]:
queries_folder = wd_path+"/queries"
from os import listdir
queries = os.listdir(queries_folder)
print(queries)

['creacion_tabla_OutputEvaluacion.sql', 'inserto_datos_en_OutputEvaluacion.sql']


In [28]:
for i in range(len(queries)):
    with open(queries_folder+'/'+queries[i], "r") as f:
        lines = f.readlines()
        for line in lines:
            print(line.strip())
        

CREATE TABLE "OutputEvaluacion" (
"field1"	INTEGER,
"anio"	INTEGER,
"cuit_impor"	REAL,
"rsocial"	TEXT,
"prove_dest"	TEXT,
"sim_sptos"	TEXT,
"porg"	INTEGER,
"origen"	TEXT,
"u_estad"	INTEGER,
"uni_estad"	TEXT,
"cant_est"	INTEGER,
"kg"	INTEGER,
"fob"	REAL,
"cif"	REAL,
"Nombre_Resultado" TEXT,
"Evaluacion_Resultados" TEXT
);
INSERT INTO OutputEvaluacion
SELECT bs.*,
CASE
WHEN pm.first_cleaning IS NULL THEN bs.prove_dest
WHEN pm.first_cleaning="Se borrÃ³ nombre en limpieza" THEN bs.prove_dest
WHEN pm.second_cleaning IS NULL THEN pm.first_cleaning
WHEN pm.second_cleaning="Se borrÃ³ nombre en limpieza" THEN pm.first_cleaning
WHEN pm.tfidf_cleaning="no name" THEN pm.second_cleaning
WHEN pm.tfidf_cleaning="Se borrÃ³ nombre en limpieza" THEN pm.second_cleaning
WHEN pm.tfidf_cleaning IS NULL  THEN pm.second_cleaning
ELSE pm.tfidf_cleaning
END as "Nombre_Resultado",
CASE
WHEN pm.first_cleaning IS NULL THEN "No_limpio"
WHEN pm.first_cleaning="Se borrÃ³ nombre en limpieza" THEN "No_limpio"
WHEN pm.s

In [21]:
#Creacion de Tabla OutputEvaluacion
run_sql_file(nombre_base, queries_folder+'/'+queries[0])

Successfully Connected to SQLite
SQLite script executed successfully
sqlite connection is closed


In [29]:
#Inserto Datos desde 
run_sql_file(nombre_base, queries_folder+'/'+queries[1])

Successfully Connected to SQLite
SQLite script executed successfully
sqlite connection is closed


In [31]:
from os import listdir
queries = os.listdir(queries_folder)
print(queries)
grpby_query = queries[1]
print(grpby_query)

['creacion_tabla_OutputEvaluacion.sql', 'groupby_Nombre_Resultado.sql', 'inserto_datos_en_OutputEvaluacion.sql']
groupby_Nombre_Resultado.sql


In [32]:
cols = retrieve_query_keys(nombre_base,queries_folder+'/'+grpby_query)
print("La query devolvió {} columnas".format(len(cols)))
rows = retrieve_query_data(nombre_base,queries_folder+'/'+grpby_query)
print("La query devolvió {} filas".format(len(rows)))

Successfully Connected to SQLite
******************************
Your query is:
SELECT Nombre_Resultado, sum(fob) as "SumaFob", sum(cif) as "SumaCif"
FROM OutputEvaluacion
GROUP BY Nombre_Resultado
ORDER BY "SumaFob" DESC
******************************
SQLite script executed successfully
sqlite connection is closed
La query devolvió 3 columnas
Successfully Connected to SQLite
******************************
Your query is:
SELECT Nombre_Resultado, sum(fob) as "SumaFob", sum(cif) as "SumaCif"
FROM OutputEvaluacion
GROUP BY Nombre_Resultado
ORDER BY "SumaFob" DESC
******************************
SQLite script executed successfully
sqlite connection is closed
La query devolvió 2118766 filas


In [ ]:
with open('fob_por_Nombre_Resultado.csv', 'w', newline='\n') as f: 
    write = csv.writer(f) 
    write.writerow(cols) 
    write.writerows(rows) 